In [1]:
!pip install pdfplumber nltk rouge_score sentence-transformers scikit-learn bert-score -q
!pip install nltk -q
!pip install pip3-autoremove -q
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124 -q
!pip install unsloth -q
!pip install --upgrade "transformers>=4.53.0" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import unsloth
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096  
dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16  
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-11 20:59:27.223790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752267567.599160      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752267567.706691      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
import torch
import pdfplumber
import os
from nltk.tokenize import sent_tokenize
import nltk
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from bert_score import score

nltk.download('punkt')

print("Using device:", "cuda" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache()

Using device: cuda


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    )

==((====))==  Unsloth 2025.7.3: Fast Mistral patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def extract_text(file_path):
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + " "
        return text.strip()
    elif file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read().strip()
    else:
        raise ValueError("Unsupported file format. Use .pdf or .txt")

In [7]:
def chunk_text(text, max_tokens=4000, overlap_tokens=200):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_token_count = 0
    
    for sentence in sentences:
        sentence_tokens = tokenizer.encode(sentence, add_special_tokens=False)
        if current_token_count + len(sentence_tokens) > max_tokens:
            chunks.append(" ".join(current_chunk))
            overlap_sentences = []
            overlap_count = 0
            for prev_sentence in current_chunk[::-1]:
                prev_tokens = len(tokenizer.encode(prev_sentence, add_special_tokens=False))
                if overlap_count + prev_tokens <= overlap_tokens:
                    overlap_sentences.append(prev_sentence)
                    overlap_count += prev_tokens
                else:
                    break
            current_chunk = overlap_sentences[::-1]
            current_token_count = overlap_count
        current_chunk.append(sentence)
        current_token_count += len(sentence_tokens)
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks if chunks else [text]

In [8]:
def get_reference_summary(document_text):

    sentences = sent_tokenize(document_text)
    if not sentences:
        return ""
    legal_terms = ["Companies Act, 2013", "confidentiality", "termination", "indemnity", "arbitration", "enforceability", "restrictive covenant", "intellectual property"]
    vectorizer = TfidfVectorizer(stop_words='english', max_features=100)
    try:
        tfidf_matrix = vectorizer.fit_transform(sentences)
        feature_names = vectorizer.get_feature_names_out()
        sentence_scores = np.sum(tfidf_matrix.toarray(), axis=1)

        for i, sentence in enumerate(sentences):
            for term in legal_terms:
                if term.lower() in sentence.lower():
                    sentence_scores[i] *= 1.5

        top_indices = np.argsort(sentence_scores)[-5:]
        top_sentences = [sentences[i] for i in sorted(top_indices)]
        return " ".join(top_sentences)
    except ValueError:

        return " ".join(sentences[:5])

In [9]:
def evaluate_summary(generated_summary, reference_summary, document_text):
    
    P, R, F1 = score([generated_summary], [document_text], lang="en", model_type="roberta-large")
    
    return {
        "bertscore_precision": P.item(),
        "bertscore_recall": R.item(),
        "bertscore_f1": F1.item()
    }

In [10]:
def summarize_text(text, role="legal_expert", strategy="zero-shot", max_summary_length=500, tokenizer=None, model=None):
    prompts = {
        "legal_expert": {
            "zero-shot": f"""
            You are a legal expert analyzing a legal document. Summarize the document in concise bullet points, verbatim including every clause referencing The Companies Act, 2013 (e.g., section 164 disqualification, board meeting provisions), all key legal terms (e.g., termination, confidentiality, indemnity, arbitration, restrictive covenant, intellectual property), potential legal risks, and enforceability. Prioritize legal precision and completeness, excluding all non-legal details.
            Document:
            {text}

            Summary:
            """,
            "chain-of-thought": f"""
            You are a legal expert analyzing a legal document. To summarize: 1) Identify the document's title and objective. 2) Verbatim include every clause explicitly referencing The Companies Act, 2013 (e.g., section 164 disqualification, board meeting provisions), and all key legal terms (e.g., termination, confidentiality, indemnity, arbitration, restrictive covenant, intellectual property), avoiding extraneous reasoning. 3) Analyze potential legal risks and enforceability. Provide a concise summary in bullet points, excluding non-essential details and ensuring legal precision.
            Document:
            {text}

            Analysis and Summary:
            """,
            "role-based": f"""
            As a senior legal expert specializing in corporate law, summarize the document in concise bullet points, verbatim including every clause referencing The Companies Act, 2013 (e.g., section 164 disqualification, board meeting provisions), all key legal terms (e.g., termination, confidentiality, indemnity, arbitration, restrictive covenant, intellectual property), potential legal risks, and enforceability. Exclude non-essential details and ensure legal precision.
            Document:
            {text}

            Summary:
            """
        },
        "corporate_executive": {
            "zero-shot": f"""
            You are a corporate executive reviewing a legal document. Summarize the document in concise bullet points, focusing only on business implications, key obligations of the parties, financial terms, and alignment with The Companies Act, 2013. Exclude non-essential details.
            Document:
            {text}

            Summary:
            """,
            "chain-of-thought": f"""
            You are a corporate executive reviewing a legal document. To summarize: 1) Identify the document's purpose and parties. 2) List business implications, key obligations, financial terms, and alignment with The Companies Act, 2013. Provide a concise summary in bullet points, excluding non-essential details.
            Document:
            {text}

            Analysis and Summary:
            """,
            "role-based": f"""
            As a corporate executive with expertise in business strategy, summarize the document in concise bullet points, focusing only on business implications, key obligations of the parties, financial terms, and alignment with The Companies Act, 2013. Exclude non-essential details.
            Document:
            {text}

            Summary:
            """
        },
        "law_student": {
            "zero-shot": f"""
            You are a corporate law student studying a legal document. Summarize the document in concise bullet points, focusing only on its structure, key legal terms, references to The Companies Act, 2013, and educational insights into its purpose and clauses. Exclude non-essential details.
            Document:
            {text}

            Summary:
            """,
            "chain-of-thought": f"""
            You are a corporate law student studying a legal document. To summarize: 1) Identify the document's structure and purpose. 2) List key legal terms, references to The Companies Act, 2013, and educational insights into its clauses. Provide a concise summary in bullet points, excluding non-essential details.
            Document:
            {text}

            Analysis and Summary:
            """,
            "role-based": f"""
            As a corporate law student eager to learn, summarize the document in concise bullet points, focusing only on its structure, key legal terms, references to The Companies Act, 2013, and educational insights into its purpose and clauses. Exclude non-essential details.
            Document:
            {text}

            Summary:
            """
        },
        "compliance_officer": {
            "zero-shot": f"""
            You are a compliance officer evaluating a legal document. Summarize the document in concise bullet points, focusing only on compliance requirements, adherence to The Companies Act, 2013, confidentiality obligations, and potential regulatory issues. Exclude non-essential details.
            Document:
            {text}

            Summary:
            """,
            "chain-of-thought": f"""
            You are a compliance officer evaluating a legal document. To summarize: 1) Identify compliance-related clauses. 2) List adherence to The Companies Act, 2013, confidentiality obligations, and potential regulatory issues. Provide a concise summary in bullet points, excluding non-essential details.
            Document:
            {text}

            Analysis and Summary:
            """,
            "role-based": f"""
            As a compliance officer specializing in regulatory oversight, summarize the document in concise bullet points, focusing only on compliance requirements, adherence to The Companies Act, 2013, confidentiality obligations, and potential regulatory issues. Exclude non-essential details.
            Document:
            {text}

            Summary:
            """
        }
    }

    if role not in prompts or strategy not in prompts[role]:
        raise ValueError(f"Invalid role '{role}' or strategy '{strategy}'")

    prompt = prompts[role][strategy]

    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to("cuda")
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_summary_length,
            temperature=0.5,
            top_k=40,
            top_p=0.9,
            do_sample=True
        )
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

        if "Summary:" in summary:
            summary = summary.split("Summary:")[1].strip()
        elif "Analysis and Summary:" in summary:
            summary = summary.split("Analysis and Summary:")[1].strip()
        else:
            summary = summary.strip()

        sentences = sent_tokenize(summary)
        doc_embedding = sentence_model.encode(text, convert_to_tensor=True)
        sentence_embeddings = sentence_model.encode(sentences, convert_to_tensor=True)
        similarities = util.cos_sim(sentence_embeddings, doc_embedding)

        legal_terms = ["Companies Act, 2013", "confidentiality", "termination", "indemnity", "arbitration", "enforceability", "restrictive covenant", "intellectual property"]
        sentence_scores = similarities.cpu().numpy().flatten()
        for i, sentence in enumerate(sentences):
            for term in legal_terms:
                if term.lower() in sentence.lower():
                    sentence_scores[i] *= 1.25  

            sentence_length = len(sentence.split())
            sentence_scores[i] /= (1 + 0.005 * max(0, sentence_length - 20))

        threshold = 0.05 if strategy == "zero-shot" else 0.1
        filtered_sentences = [sentences[i] for i in range(len(sentences)) if sentence_scores[i] > threshold]
        return " ".join(filtered_sentences) if filtered_sentences else summary
    except Exception as e:
        print(f"Error generating summary for role {role} and strategy {strategy}: {e}")
        return f"Error: Failed to generate summary for role {role} and strategy {strategy}"

In [11]:
!pip install gradio -q

In [12]:
import gradio as gr

def process_pdf(file, role):
    if not file:
        return "Error: No file uploaded."
    
    # Extract text from uploaded PDF
    document_text = extract_text(file)
    if not document_text or document_text.startswith("Error"):
        return document_text

    # Chunk the text
    chunks = chunk_text(document_text)
    
    # Get reference summary
    reference_summary = get_reference_summary(document_text)
    
    # Process summaries for all strategies
    strategies = ["zero-shot", "chain-of-thought", "role-based"]
    summaries = {}
    evaluations = {}
    
    for strategy in strategies:
        chunk_summaries = []
        for i, chunk in enumerate(chunks):
            summary = summarize_text(
                text=chunk,
                role=role,
                strategy=strategy,
                tokenizer=tokenizer,
                model=model,
            )
            chunk_summaries.append(summary)
        combined_summary = " ".join(chunk_summaries)
        summaries[strategy] = combined_summary
        evaluations[strategy] = evaluate_summary(combined_summary, reference_summary, document_text)
    
    # Format output
    output = ""
    for strategy in strategies:
        output += f"\n{role.replace('_', ' ').title()} - {strategy.replace('-', ' ').title()} Summary:\n"
        output += summaries[strategy] + "\n"
        output += f"\nEvaluation Metrics for {strategy.replace('-', ' ').title()} Summary:\n"
        eval_metrics = evaluations[strategy]
        output += f"BERTScore Precision: {eval_metrics['bertscore_precision']:.4f}\n"
        output += f"BERTScore Recall: {eval_metrics['bertscore_recall']:.4f}\n"
        output += f"BERTScore F1: {eval_metrics['bertscore_f1']:.4f}\n"
    
    return output

iface = gr.Interface(
    fn=process_pdf,
    inputs=[
        gr.File(label="Upload PDF File", file_types=[".pdf"]),
        gr.Dropdown(
            choices=["legal_expert", "corporate_executive", "law_student", "compliance_officer"],
            label="Select Role",
            value="legal_expert"
        )
    ],
    outputs=gr.Textbox(label="Summary and Evaluation"),
    title="PDF Document Summarizer",
    description="Upload a PDF file and select a role to generate summaries with different strategies."
)

# Launch the interface
iface.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://c6497b2c203ef6f883.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
